# Classifying White Blood Cells

### Introduction

An important problem in blood diagnostics is classifying different types of blood cells. In this notebook, we will attempt to train a classifier to predict the type of a blood cell given a dyed picture of it.

### Data

We have 351 pictures of dyed white blood cells along with labels of what type of blood cell they are. Below is an example of each of the types of blood cells in our dataset.

##### Basophil
![Basophil](Basophil.jpg)

#### Eosinophil
![Eosinophil](Eosinophil.jpg)

#### Lymphocyte
![Lymphocyte](Lymphocyte.jpg)

#### Monocyte
![Monocyte](Monocyte.jpg)

#### Neutrophil
![Neutrophil](Neutrophil.jpg)

### Methodology

We use a simple LeNet architecture trained on 280 training samples with image augmentation. Our augmentation techniques include rotations, shifts, and zooms.

We validate our results against 70 samples.

### Results

We obtain an accuracy of 93% on this validation set with the following confusion matrix:

![Confusion Matrix](confusion_matrix.png)


### Conclusion

I think these results show that it is possible to train a strong classifier for WBCs that can approach really strong accuracies with more and more data.

### Next Steps

It's hard to try and improve the results with so little validation data. Fixing one prediction here or there would lead to a big change in accuracy etc. The next steps should be to augment our training and test sets with further image augmentation techniques or just more real data.

Here are some potential techniques to improve accuracy:

* Use color thresholding to find the center of each WBC and crop the images to be centered around those areas.
* Use VGG net etc. and retrain the last few layers.
* Balance out the dataset through image augmentation.


# Code

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
import csv
import cv2
import scipy


ModuleNotFoundError: No module named 'keras'

In [ ]:
num_classes = 5
epochs = 20
# BASE_PATH = '/home/ec2-user/cell_classifier/'
BASE_PATH = '../'
batch_size = 32

In [ ]:
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

    return model

In [ ]:
def get_filename_for_index(index):
    PREFIX = 'images/BloodImage_'
    num_zeros = 5 - len(index)
    path = '0' * num_zeros + index
    return PREFIX + path + '.jpg'

In [ ]:
reader = csv.reader(open(BASE_PATH + 'labels.csv'))
# skip the header
next(reader)

X = []
y = []

for row in reader:
    label = row[2]
    if len(label) > 0 and label.find(',') == -1:
        filename = get_filename_for_index(row[1])
        img_file = cv2.imread(BASE_PATH + filename)
        if img_file is not None:
            img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
            img_arr = np.asarray(img_file)
            img_arr = apply_color_mask(img_arr)
            X.append(img_arr)
            y.append(label)


X = np.asarray(X)
y = np.asarray(y)

encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

y = np_utils.to_categorical(encoded_y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
train_generator = datagen.flow(
        X_train,
        y_train,
        batch_size=batch_size)

validation_generator = datagen.flow(
        X_test,
        y_test,
        batch_size=batch_size)

In [ ]:
model = get_model()

# fits the model on batches with real-time data augmentation:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(X_train),
    validation_data=validation_generator,
    validation_steps=len(X_test),
    epochs=epochs)
model.save_weights('mask_1.h5')  # always save your weights after training or during training

# Load Model

In [ ]:
model = get_model()
model.load_weights('first_try.h5')

# Accuracy

In [ ]:
print('Predicting on test data')
y_pred = np.rint(model.predict(X_test))

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_unencoded = np.argmax(y_pred, axis=1)
y_test_unencoded = np.argmax(y_test, axis=1)

print(confusion_matrix(y_test_unencoded, y_pred_unencoded))

In [ ]:
encoder.inverse_transform([0, 1, 2, 3, 4])

# Eosinophils

In [ ]:
eosinophils = np.where(y_pred[:,1] == 1)

In [ ]:
eosinophils

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[11],y_pred[11]))
print(y_test[11])
print(y_pred[11])
imshow(X_test[11])

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[11],y_pred[11]))
print(y_test[15])
print(y_pred[15])
imshow(X_test[15])

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[11],y_pred[11]))
print(y_test[16])
print(y_pred[16])
imshow(X_test[16])

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[11],y_pred[11]))
print(y_test[11])
print(y_pred[11])
imshow(X_test[11])

# Lymphocytes

In [ ]:
lymphocytes = np.where(y_pred[:,2] == 1)
print(lymphocytes)

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[22],y_pred[22]))
print(y_test[22])
print(y_pred[22])
imshow(X_test[22])

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[22],y_pred[22]))
print(y_test[32])
print(y_pred[32])
imshow(X_test[32])

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[22],y_pred[22]))
print(y_test[60])
print(y_pred[60])
imshow(X_test[60])

# Monocytes

In [ ]:
monocytes = np.where(y_pred[:,3] == 1)
print(monocytes)

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[6],y_pred[6]))
print(y_test[6])
print(y_pred[6])
imshow(X_test[6])

In [ ]:
print("IS CORRECT?", np.array_equal(y_test[28],y_pred[28]))
print(y_test[28])
print(y_pred[28])
imshow(X_test[28])